In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [24]:
df_train= pd.read_csv('../input/train.csv')
df_test= pd.read_csv('../input/test.csv')
df_test1= pd.read_csv('../input/test.csv')


In [25]:
df_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
df_train['Deck']= df_train.Cabin[df_train.Cabin.isna() == False].str.slice(0,1)
df_test['Deck']= df_test.Cabin[df_test.Cabin.isna() == False].str.slice(0,1)

In [27]:
df_train.drop(['PassengerId','Cabin','Name', 'Ticket'], axis=1, inplace=True)
df_test.drop(['PassengerId','Cabin','Name', 'Ticket'], axis=1, inplace=True)

In [28]:
df_train_encoded= pd.get_dummies(df_train)
df_train_encoded.Age.fillna(df_train_encoded.Age.mean(), inplace=True)
df_train_encoded.drop('Deck_T', axis=1, inplace=True)

df_test_encoded= pd.get_dummies(df_test)
df_test_encoded.Age.fillna(df_train_encoded.Age.mean(), inplace=True)
df_test_encoded.Fare.fillna(df_train_encoded.Fare.mean(), inplace=True)

In [29]:

# df_encoded= pd.get_dummies(df_train.Deck)

# df_train_encoded= pd.concat([df_train, df_encoded], axis=1)

# df_train['Section_Number'] = df_train['Cabin'].str.replace('([A-Z]+)', '')
# df_train['Section_Letter'] = df_train['Cabin'].str.extract('([A-Z]+)')
# df_train.Cabin.str.findall(r'(\d+\.?\d+)')

In [30]:
df_train_encoded.corr()['Survived']

Survived      1.000000
Pclass       -0.338481
Age          -0.069809
SibSp        -0.035322
Parch         0.081629
Fare          0.257307
Sex_female    0.543351
Sex_male     -0.543351
Embarked_C    0.168240
Embarked_Q    0.003650
Embarked_S   -0.155660
Deck_A        0.022287
Deck_B        0.175095
Deck_C        0.114652
Deck_D        0.150716
Deck_E        0.145321
Deck_F        0.057935
Deck_G        0.016040
Name: Survived, dtype: float64

In [31]:
X_train= df_train_encoded.drop('Survived', axis=1)
y_train= df_train_encoded.Survived


In [32]:
# # Binary encoding
# import category_encoders as ce

# df_train_binary = df_train.copy()

# encoder = ce.BinaryEncoder(cols=['Cabin'])
# df_binary = encoder.fit_transform(df_train_binary)



In [33]:
# Forward feature selection
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# Build step forward feature selection
sfs1 = sfs(forest,
           k_features=8,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS

sfs1 = sfs1.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:   20.6s finished

[2019-07-25 22:51:42] Features: 1/8 -- score: 0.7867150249291879[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:   18.3s finished

[2019-07-25 22:52:01] Features: 2/8 -- score: 0.7968400804741635[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   17.6s finished

[2019-07-25 22:52:18] Features: 3/8 -- score: 0.7968716786055118[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [34]:
feat_cols= list(sfs1.k_feature_names_)
X_train.loc[:,feat_cols].head()

,Pclass,Age,SibSp,Fare,Sex_female,Deck_B,Deck_D,Deck_E
0,3,22.0,1,7.2500,0,0,0,0
1,1,38.0,1,71.2833,1,0,0,0
2,3,26.0,0,7.9250,1,0,0,0
3,1,35.0,1,53.1000,1,0,0,0
4,3,35.0,0,8.0500,0,0,0,0


In [35]:
rf = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=4)
rf.fit(X_train.loc[:, feat_cols], y_train)

y_train_pred = rf.predict(X_train.loc[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = rf.predict(df_test_encoded.loc[:, feat_cols])
# print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred))

Training accuracy on selected features: 0.840


In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [37]:
from sklearn.ensemble import RandomForestRegressor

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train.loc[:,feat_cols], y_train)



Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [38]:
y_pred_forest_grid= rf_random.predict(df_test_encoded.loc[:, feat_cols])

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()
param_grid = {'penalty':['l1','l2']}
GL = GridSearchCV(log,param_grid,cv=4)
GL.fit(X_train, y_train)
GL.score(X_train, y_train)

y_pred= GL.predict(df_test_encoded)



/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [40]:
pred= pd.DataFrame({'PassengerId':df_test1.PassengerId, 'Survived': y_pred_forest_grid})
pred.to_csv('prediction.csv', index=False)